In [18]:
import sys, string
sys.path.append('/home/grg/git/thesaurus/')
import thesaurus as th
jf = '/home/grg/git/thesaurus/alfa_dwi_pipeline_aug2016.json'
dd = '/home/grg/data/AmylStaging'
json_io = '/home/grg/git/thesaurus/alfa_dwi_pipeline_io_aug2016.json'
step = 'ants_dwi_to_t1'

subjects = [string.atoi(e) for e in open('/tmp/amyloid_subjects.txt').read().split('\n') if e != '']
failed = [12118, 13041, 55854, 10162, 10515, 10627, 10649, 10711, 10949, 11262, 11514, 11745, 12279, 12741, 12991, 13035, 13105, 13107, 13193, 13259, 
             21136, 21138, 44008, 44301, 44396, 66008, 66027, 66100, 66162, 66301, 77012, 77164]
subjects = [e for e in subjects if not e in failed]

a = th.ALFAHelper(directory=dd, jsonfile=json_io)

cmds = []
for s in subjects:  
    cmd = a.parse_command(str(s), step, jsonfile=jf)
    cmds.append(cmd)
 
# Writing commands in a textfile
fp1 = '/tmp/%s.sh'%step
fp2 = '/tmp/%s.workflow'%step
w = open(fp1, 'w')
for e in cmds:
    w.write('%s\n'%e)
w.close()

import sys
sys.path.append('/home/grg/git/alfa')
import create_workflow as cw
names = [str(e) for e in subjects]

cw.create_workflow(fp1, fp2, names=names, verbose=True)

[['ANTS', '3', '-m', 'CC[', '/home/grg/data/AmylStaging/10019/T1/10019_mabonlm_nobias_spm_c2_d2.nii.gz,', '/home/grg/data/AmylStaging/10019/DWI/10019_B0_brain_seg_1.nii.gz,', '1,', '4]', '-r', 'Gauss[0,3]', '-t', 'Elast[1.5]', '-i', '30x20x10', '-o', '/home/grg/data/AmylStaging/10019/DWI/ants_elast_dwi-to-t1_.nii.gz'], ['ANTS', '3', '-m', 'CC[', '/home/grg/data/AmylStaging/10044/T1/10044_mabonlm_nobias_spm_c2_d2.nii.gz,', '/home/grg/data/AmylStaging/10044/DWI/10044_B0_brain_seg_1.nii.gz,', '1,', '4]', '-r', 'Gauss[0,3]', '-t', 'Elast[1.5]', '-i', '30x20x10', '-o', '/home/grg/data/AmylStaging/10044/DWI/ants_elast_dwi-to-t1_.nii.gz'], ['ANTS', '3', '-m', 'CC[', '/home/grg/data/AmylStaging/10065/T1/10065_mabonlm_nobias_spm_c2_d2.nii.gz,', '/home/grg/data/AmylStaging/10065/DWI/10065_B0_brain_seg_1.nii.gz,', '1,', '4]', '-r', 'Gauss[0,3]', '-t', 'Elast[1.5]', '-i', '30x20x10', '-o', '/home/grg/data/AmylStaging/10065/DWI/ants_elast_dwi-to-t1_.nii.gz'], ['ANTS', '3', '-m', 'CC[', '/home/grg/d

In [4]:
cmds[0]

u'ANTS 3 -m CC[ /home/grg/data/AmylStaging/10019/T1/10019_mabonlm_nobias_spm_c2_d2.nii.gz, /home/grg/data/AmylStaging/10019/DWI/10019_B0_brain_seg_1.nii.gz, 1, 4] -r Gauss[0,3] -t Elast[1.5] -i 30x20x10 -o /home/grg/data/AmylStaging/10019/DWI/ants_elast_dwi-to-t1_.nii.gz'

### ANTs using Nipype

In [33]:
dd = '/tmp/nipype'
import os.path as osp
from nipype.interfaces.ants import ANTS
reg = ANTS()
reg.inputs.fixed_image = osp.join(dd, '55854_mabonlm_nobias_spm_c2.nii')
reg.inputs.moving_image = osp.join(dd, '55854_B0_brain_seg_1.nii.gz')
reg.inputs.output_transform_prefix = 'ants_elast_dwi-to-t1_.nii.gz'
reg.inputs.dimension = 3
reg.inputs.metric = ['CC']*2
reg.inputs.radius = [4]*2
reg.inputs.transformation_model = 'Elast'
reg.inputs.gradient_step_length = 1.5
reg.inputs.number_of_iterations = [30,20,10]
reg.inputs.regularization = 'Gauss'
reg.inputs.regularization_deformation_field_sigma = 3
reg.inputs.regularization_gradient_field_sigma = 0
reg.cmdline


u'ANTS 3 --image-metric CC[ /tmp/nipype/55854_mabonlm_nobias_spm_c2.nii, /tmp/nipype/55854_B0_brain_seg_1.nii.gz, 1, 4 ] --number-of-iterations 30x20x10 --output-naming ants_elast_dwi-to-t1_.nii.gz --regularization Gauss[0.0,3.0] --transformation-model Elast[1.5] --use-Histogram-Matching 0'

In [27]:
from nipype.interfaces.ants import WarpImageMultiTransform
w = WarpImageMultiTransform()
w.inputs

In [32]:
dd = '/tmp/nipype'
import os.path as osp
from nipype.interfaces.ants import ANTS
reg = Registration()

reg.inputs.fixed_image = osp.join(dd, '55854_mabonlm_nobias_spm_c2.nii')
reg.inputs.moving_image = osp.join(dd, '55854_B0_brain_seg_1.nii.gz')
reg.inputs.output_transform_prefix = 'ants_elast_dwi-to-t1_'
reg.inputs.initial_moving_transform = 'trans.mat'
reg.inputs.transforms = ['Affine', 'SyN']
reg.inputs.transform_parameters = [(2.0,), (0.25, 3.0, 0.0)]
reg.inputs.number_of_iterations = [[1500, 200], [100, 50, 30]]
reg.inputs.dimension = 3
reg.inputs.write_composite_transform = True
reg.inputs.collapse_output_transforms = False
reg.inputs.initialize_transforms_per_stage = False
reg.inputs.metric = ['CC']*2
reg.inputs.metric_weight = [1]*2 # Default (value ignored currently by ANTs)
reg.inputs.radius_or_number_of_bins = [4]*2
reg.inputs.convergence_threshold = [1.e-8, 1.e-9]
reg.inputs.convergence_window_size = [20]*2
reg.inputs.smoothing_sigmas = [[1,0], [2,1,0]]
reg.inputs.sigma_units = ['vox'] * 2
reg.inputs.shrink_factors = [[2,1], [3,2,1]]
reg.inputs.use_estimate_learning_rate_once = [True, True]
reg.inputs.use_histogram_matching = [True, True] # This is the default
reg.inputs.output_warped_image = '55854_B0_warped_image.nii.gz'
reg.cmdline

u'antsRegistration --collapse-output-transforms 0 --dimensionality 3 --initial-moving-transform [ trans.mat, 0 ] --initialize-transforms-per-stage 0 --interpolation Linear --output [ ants_elast_dwi-to-t1_, 55854_B0_warped_image.nii.gz ] --transform Affine[ 2.0 ] --metric CC[ /tmp/nipype/55854_mabonlm_nobias_spm_c2.nii, /tmp/nipype/55854_B0_brain_seg_1.nii.gz, 1, 4 ] --convergence [ 1500x200, 1e-08, 20 ] --smoothing-sigmas 1.0x0.0vox --shrink-factors 2x1 --use-estimate-learning-rate-once 1 --use-histogram-matching 1 --transform SyN[ 0.25, 3.0, 0.0 ] --metric CC[ /tmp/nipype/55854_mabonlm_nobias_spm_c2.nii, /tmp/nipype/55854_B0_brain_seg_1.nii.gz, 1, 4 ] --convergence [ 100x50x30, 1e-09, 20 ] --smoothing-sigmas 2.0x1.0x0.0vox --shrink-factors 3x2x1 --use-estimate-learning-rate-once 1 --use-histogram-matching 1 --winsorize-image-intensities [ 0.0, 1.0 ]  --write-composite-transform 1'

In [19]:
subjects = [string.atoi(e) for e in open('/tmp/amyloid_subjects.txt').read().split('\n') if e != '']
cmds = []
names = []
dd = '/home/grg/data/AmylStaging/'
#for subject in subjects:
#    names.append('%s_fslbet'%subject)
#    cmd = 'fsl5.0-bet %s%s/T1/%s_mabonlm_nobias.nii %s%s/T1/%s_mabonlm_nobias_brain.nii.gz -B -f 0.2 -g 0 -m'%(dd, subject, subject, dd, subject, subject)
#    cmds.append(cmd)

for subject in subjects:
    names.append('%s_ants'%subject)
    #reg = ANTS()
    #reg.inputs.fixed_image = osp.join(dd, str(subject), 'T1', '%s_mabonlm_nobias_brain.nii.gz'%subject)
    #reg.inputs.moving_image = osp.join(dd, str(subject), 'T1', '%s_B0_brain.nii'%subject)
    #reg.inputs.output_transform_prefix = 'ants_elast_dwi-to-t1_.nii.gz'
    #reg.inputs.dimension = 3
    #reg.inputs.metric = ['CC']*2
    #reg.inputs.radius = [4]*2
    #reg.inputs.transformation_model = 'Elast'
    #reg.inputs.gradient_step_length = 1.5
    #reg.inputs.number_of_iterations = [30,20,10]
    #reg.inputs.regularization = 'Gauss'
    #reg.inputs.regularization_deformation_field_sigma = 3
    #reg.inputs.regularization_gradient_field_sigma = 0
    #cmds.append(reg.cmdline)
    cmd = 'ANTS 3 --image-metric MI[ /home/grg/data/AmylStaging/55854/T1/55854_mabonlm_nobias_brain.nii.gz, /home/grg/data/AmylStaging/55854/DWI/55854_B0.nii.gz, 1, 4 ] --number-of-iterations 30x20x10 --output-naming /home/grg/data/AmylStaging/55854/DWI/ants_elast_MI_dwi-to-t1_.nii.gz --regularization Gauss[0.0,3.0] --transformation-model Elast[1.5] --use-Histogram-Matching 0'
    cmd = cmd.replace('55854', str(subject))
    cmds.append(cmd)

step = 'ants_MI'
# Writing commands in a textfile
fp1 = '/tmp/%s.sh'%step
fp2 = '/tmp/%s.workflow'%step
w = open(fp1, 'w')
for e in cmds:
    w.write('%s\n'%e)
w.close()

import sys
sys.path.append('/home/grg/git/alfa')
import create_workflow as cw

cw.create_workflow(fp1, fp2, names=names, verbose=True)

[['ANTS', '3', '--image-metric', 'MI[', '/home/grg/data/AmylStaging/10019/T1/10019_mabonlm_nobias_brain.nii.gz,', '/home/grg/data/AmylStaging/10019/DWI/10019_B0.nii.gz,', '1,', '4', ']', '--number-of-iterations', '30x20x10', '--output-naming', '/home/grg/data/AmylStaging/10019/DWI/ants_elast_MI_dwi-to-t1_.nii.gz', '--regularization', 'Gauss[0.0,3.0]', '--transformation-model', 'Elast[1.5]', '--use-Histogram-Matching', '0'], ['ANTS', '3', '--image-metric', 'MI[', '/home/grg/data/AmylStaging/10044/T1/10044_mabonlm_nobias_brain.nii.gz,', '/home/grg/data/AmylStaging/10044/DWI/10044_B0.nii.gz,', '1,', '4', ']', '--number-of-iterations', '30x20x10', '--output-naming', '/home/grg/data/AmylStaging/10044/DWI/ants_elast_MI_dwi-to-t1_.nii.gz', '--regularization', 'Gauss[0.0,3.0]', '--transformation-model', 'Elast[1.5]', '--use-Histogram-Matching', '0'], ['ANTS', '3', '--image-metric', 'MI[', '/home/grg/data/AmylStaging/10065/T1/10065_mabonlm_nobias_brain.nii.gz,', '/home/grg/data/AmylStaging/1006

In [10]:
import sys, string
sys.path.append('/home/grg/git/thesaurus/')
import thesaurus as th
jf = '/home/grg/git/thesaurus/alfa_dwi_pipeline_aug2016.json'
dd = '/home/grg/data/AmylStaging'
json_io = '/home/grg/git/thesaurus/alfa_dwi_pipeline_io_aug2016.json'
step = 'warp_fa_to_t1'

subjects = [string.atoi(e) for e in open('/tmp/amyloid_subjects.txt').read().split('\n') if e != '']
failed = [12118, 13041, 55854, 10162, 10515, 10627, 10649, 10711, 10949, 11262, 11514, 11745, 12279, 12741, 12991, 13035, 13105, 13107, 13193, 13259, 
             21136, 21138, 44008, 44301, 44396, 66008, 66027, 66100, 66162, 66301, 77012, 77164]
#subjects = [e for e in subjects if not e in failed]
subjects = [10711, 11514,11745,12279,12741,55854]

a = th.ALFAHelper(directory=dd, jsonfile=json_io)

cmds = []
for s in subjects:  
    cmd = a.parse_command(str(s), step, jsonfile=jf)
    cmds.append(cmd)
 
# Writing commands in a textfile
fp1 = '/tmp/%s.sh'%step
fp2 = '/tmp/%s.workflow'%step
w = open(fp1, 'w')
for e in cmds:
    w.write('%s\n'%e)
w.close()

import sys
sys.path.append('/home/grg/git/alfa')
import create_workflow as cw
names = [str(e) for e in subjects]

cw.create_workflow(fp1, fp2, names=names, verbose=True)

[['WarpImageMultiTransform', '3', '/home/grg/data/AmylStaging/10711/DWI/10711_FA.nii.gz', '/home/grg/data/AmylStaging/10711/DWI/10711_FA_t1space.nii.gz', '-R', '/home/grg/data/AmylStaging/10711/T1/10711_mabonlm_nobias.nii', '/home/grg/data/AmylStaging/10711/DWI/ants_elast_dwi-to-t1_Warp.nii.gz', '/home/grg/data/AmylStaging/10711/DWI/ants_elast_dwi-to-t1_Affine.txt'], ['WarpImageMultiTransform', '3', '/home/grg/data/AmylStaging/11514/DWI/11514_FA.nii.gz', '/home/grg/data/AmylStaging/11514/DWI/11514_FA_t1space.nii.gz', '-R', '/home/grg/data/AmylStaging/11514/T1/11514_mabonlm_nobias.nii', '/home/grg/data/AmylStaging/11514/DWI/ants_elast_dwi-to-t1_Warp.nii.gz', '/home/grg/data/AmylStaging/11514/DWI/ants_elast_dwi-to-t1_Affine.txt'], ['WarpImageMultiTransform', '3', '/home/grg/data/AmylStaging/11745/DWI/11745_FA.nii.gz', '/home/grg/data/AmylStaging/11745/DWI/11745_FA_t1space.nii.gz', '-R', '/home/grg/data/AmylStaging/11745/T1/11745_mabonlm_nobias.nii', '/home/grg/data/AmylStaging/11745/DWI/

In [17]:
from glob import glob
files = glob('/home/grg/data/AmylStaging/*/T1/ants_elast_dwi-to-t1*')
for each in files:
    fp = each.replace('/T1/', '/DWI/')
    cmd = 'mv %s %s'%(each, fp)
    print cmd
    #os.system(cmd)

mv /home/grg/data/AmylStaging/66100/T1/ants_elast_dwi-to-t1_Warp.nii.gz /home/grg/data/AmylStaging/66100/DWI/ants_elast_dwi-to-t1_Warp.nii.gz
mv /home/grg/data/AmylStaging/66100/T1/ants_elast_dwi-to-t1_Affine.txt /home/grg/data/AmylStaging/66100/DWI/ants_elast_dwi-to-t1_Affine.txt
mv /home/grg/data/AmylStaging/44304/T1/ants_elast_dwi-to-t1_Warp.nii.gz /home/grg/data/AmylStaging/44304/DWI/ants_elast_dwi-to-t1_Warp.nii.gz
mv /home/grg/data/AmylStaging/44304/T1/ants_elast_dwi-to-t1_Affine.txt /home/grg/data/AmylStaging/44304/DWI/ants_elast_dwi-to-t1_Affine.txt
mv /home/grg/data/AmylStaging/10225/T1/ants_elast_dwi-to-t1_Warp.nii.gz /home/grg/data/AmylStaging/10225/DWI/ants_elast_dwi-to-t1_Warp.nii.gz
mv /home/grg/data/AmylStaging/10225/T1/ants_elast_dwi-to-t1_Affine.txt /home/grg/data/AmylStaging/10225/DWI/ants_elast_dwi-to-t1_Affine.txt
mv /home/grg/data/AmylStaging/10162/T1/ants_elast_dwi-to-t1_Warp.nii.gz /home/grg/data/AmylStaging/10162/DWI/ants_elast_dwi-to-t1_Warp.nii.gz
mv /home/grg

In [16]:
import os